### 프로젝트 구조

In [ ]:
banking_system/
│
├── models/
│   ├── transaction.py
│   ├── account.py
│   └── user.py
│
├── services/
│   └── banking_service.py
│
├── utils/
│   ├── decorators.py
│   └── exceptions.py
│
└── main.py

#### 과제 1: Transaction 클래스 구현
파일명: banking_system/models/transaction.py

<b>해결해야 할 과제 및 요구 사항:</b>

1. 거래(Transaction) 클래스를 정의하고, transaction_type, amount, balance 속성을 초기화하는 생성자를 구현합니다.
2. 거래 정보를 문자열로 반환하는 __str__ 메서드를 구현합니다.
3. 거래 정보를 튜플로 반환하는 to_tuple 메서드를 구현합니다.

<b>변수 컨벤션:</b>

- transaction_type: 거래 유형을 나타내는 문자열 (예: "입금", "출금")
- amount: 거래 금액을 나타내는 정수
- balance: 거래 후 잔고를 나타내는 정수

In [ ]:
class Transaction:
    def __init__(self, transaction_type: str, amount: int, balance: int) -> None:
        self.transaction_type = transaction_type
        self.amount = amount
        self.balance = balance

    def __str__(self) -> str:
        return f"{self.transaction_type}: {self.amount}원, 잔고: {self.balance}원"

    def to_tuple(self) -> tuple:
        return self.transaction_type, self.amount, self.balance

#### 과제 2: Account 클래스 구현
파일명: banking_system/models/account.py

<b>해결해야 할 과제 및 요구 사항:</b>

1. __balance와 transactions 리스트를 초기화하는 생성자를 구현합니다.
2. 입금을 위한 deposit 메서드를 구현합니다.
3. 출금을 위한 withdraw 메서드를 구현합니다.
4. 잔고를 반환하는 get_balance 메서드를 구현합니다.
5. 거래 내역을 반환하는 get_transactions 메서드를 구현합니다.
6. 클래스 변수 bank_name와 클래스 메소드 get_bank_name, set_bank_name을 구현합니다.

<b>변수 컨벤션:</b>

- __balance: 계좌 잔고를 나타내는 프라이빗 정수 변수
- transactions: 거래 내역을 저장하는 리스트
- bank_name: 은행 이름을 나타내는 클래스 변수 문자열
- amount: 입금 또는 출금 금액을 나타내는 정수

In [ ]:
from banking_system.models.transaction import Transaction
from banking_system.utils.decorators import validate_transaction
from banking_system.utils.exceptions import InsufficientFundsError, NegativeAmountError

class Account:
    bank_name = "MyBank"  # 클래스 변수
    
    def __init__(self) -> None:
        self.__balance = 1000
        self.transactions = []

    @validate_transaction
    def deposit(self, amount: int) -> None:
        if amount <= 0:
            raise NegativeAmountError()
        self.__balance += amount
        self.transactions.append(Transaction("입금", amount, self.__balance))

    @validate_transaction
    def withdraw(self, amount: int) -> None:
        if amount <= 0:
            raise NegativeAmountError()
        if amount > self.__balance:
            raise InsufficientFundsError(self.__balance)
        self.__balance -= amount
        self.transactions.append(Transaction("출금", amount, self.__balance))

    def get_balance(self) -> int:
        return self.__balance

    def get_transactions(self) -> list:
        return self.transactions

    @classmethod
    def get_bank_name(cls) -> str:
        return cls.bank_name

    @classmethod
    def set_bank_name(cls, name: str) -> None:
        cls.bank_name = name


#### 과제 3: User 클래스 구현
파일명: banking_system/models/user.py

<b>해결해야 할 과제 및 요구 사항:</b>

username과 account를 초기화하는 생성자를 구현합니다.

<b>변수 컨벤션:</b>

username: 사용자의 이름을 나타내는 문자열
account: 사용자의 계좌를 나타내는 Account 객체

In [ ]:
from banking_system.models.account import Account

class User:
    def __init__(self, username: str) -> None:
        self.username = username
        self.account = Account()

#### 과제 4: BankingService 클래스 구현
파일명: banking_system/services/banking_service.py

<b>해결해야 할 과제 및 요구 사항:</b>

1. 사용자 목록을 초기화하는 생성자를 구현합니다.
2. 사용자를 추가하는 add_user 메서드를 구현합니다.
3. 사용자를 찾는 find_user 메서드를 구현합니다.
4. 사용자 메뉴를 제공하는 user_menu 메서드를 구현합니다.

<b>변수 컨벤션:</b>

- users: 사용자 목록을 저장하는 리스트
- username: 사용자의 이름을 나타내는 문자열
- user: User 객체를 나타내는 변수
- amount: 입금 또는 출금 금액을 나타내는 정수
- choice: 사용자의 선택을 나타내는 문자열

In [ ]:
from banking_system.models.user import User
from banking_system.utils.exceptions import UserNotFoundError

class BankingService:
    def __init__(self) -> None:
        self.users = []

    def add_user(self, username: str) -> None:
        self.users.append(User(username))

    def find_user(self, username: str) -> User:
        for user in self.users:
            if user.username == username:
                return user
        raise UserNotFoundError(username)

    def user_menu(self, username: str) -> None:
        user = self.find_user(username)

        while True:
            try:
                choice = input("원하는 작업을 선택하세요 (1: 입금, 2: 출금, 3: 잔고 확인, 4: 거래 내역, 5: 종료): ")
                if choice == '1':
                    amount = int(input("입금할 금액을 입력하세요: "))
                    user.account.deposit(amount)
                elif choice == '2':
                    amount = int(input("출금할 금액을 입력하세요: "))
                    user.account.withdraw(amount)
                elif choice == '3':
                    print(f"현재 잔고는 {user.account.get_balance()}원 입니다.")
                elif choice == '4':
                    for transaction in user.account.get_transactions():
                        print(transaction)
                elif choice == '5':
                    break
                else:
                    print("잘못된 입력입니다. 다시 시도하세요.")
            except ValueError as e:
                print(f"잘못된 입력입니다: {e}")
            except InsufficientFundsError as e:
                print(f"오류: {e}")
            except NegativeAmountError as e:
                print(f"오류: {e}")
            except UserNotFoundError as e:
                print(f"오류: {e}")

#### 과제 5: 데코레이터 및 예외 처리 구현
파일명: banking_system/utils/decorators.py, banking_system/utils/exceptions.py

<b>해결해야 할 과제 및 요구 사항:</b>

1. validate_transaction 데코레이터를 작성하여 금액이 0보다 큰지 확인합니다.
2. 사용자 정의 예외 클래스 InsufficientFundsError, NegativeAmountError, UserNotFoundError를 작성합니다.

##### 문제 5.1: 데코레이터 구현

- validate_transaction 데코레이터를 작성하여 금액이 0보다 큰지 확인합니다.
- 타입 힌팅을 사용하여 함수의 매개변수와 반환 타입을 지정하세요.
    - 함수 시그니처: validate_transaction(func: Callable) -> Callable

In [ ]:
from typing import Callable

def validate_transaction(func: Callable) -> Callable:
    def wrapper(self, amount: int) -> None:
        if amount <= 0:
            raise ValueError("금액은 0보다 커야 합니다.")
        return func(self, amount)
    return wrapper

#### 문제 5.2: 사용자 정의 예외 클래스 구현

- 사용자 정의 예외 클래스 InsufficientFundsError, NegativeAmountError, UserNotFoundError를 작성하세요.
- 각 예외 클래스에 타입 힌팅을 사용하여 생성자의 매개변수와 반환 타입을 지정하세요.
    - 함수 시그니처:
        - InsufficientFundsError.__init__(self, balance: int) -> None
        - NegativeAmountError.__init__(self) -> None
        - UserNotFoundError.__init__(self, username: str) -> None

In [ ]:
class InsufficientFundsError(Exception):
    def __init__(self, balance: int) -> None:
        super().__init__(f"잔액이 부족합니다. 현재 잔고: {balance}원")

class NegativeAmountError(Exception):
    def __init__(self) -> None:
        super().__init__("음수 금액은 허용되지 않습니다.")

class UserNotFoundError(Exception):
    def __init__(self, username: str) -> None:
        super().__init__(f"사용자를 찾을 수 없습니다: {username}")

#### 과제 6: 메인 함수 구현
파일명: banking_system/main.py

<b>해결해야 할 과제 및 요구 사항:</b>

1. BankingService 인스턴스를 생성합니다.
2. 사용자로부터 입력을 받아 사용자 추가 및 찾기 기능을 제공합니다.
3. 사용자 메뉴를 통해 입금, 출금, 잔고 확인, 거래 내역 기능을 실행할 수 있도록 합니다.

##### 문제 6.1: 메인 함수 구현

- BankingService 인스턴스를 생성하세요.
- 사용자로부터 입력을 받아 사용자 추가 및 찾기 기능을 구현하세요.
- 사용자 메뉴를 통해 입금, 출금, 잔고 확인, 거래 내역 기능을 실행할 수 있도록 구현하세요.
- 타입 힌팅을 사용하여 함수의 매개변수와 반환 타입을 지정하세요.
    - 함수 시그니처: main() -> None

In [ ]:
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath(__file__)))

from banking_system.services.banking_service import BankingService
from banking_system.utils.exceptions import InsufficientFundsError, NegativeAmountError, UserNotFoundError

def main() -> None:
    system = BankingService()

    while True:
        try:
            choice = input("1: 사용자 추가, 2: 사용자 찾기, 3: 종료: ")
            if choice == '1':
                username = input("사용자 이름을 입력하세요: ")
                system.add_user(username)
            elif choice == '2':
                username = input("사용자 이름을 입력하세요: ")
                system.user_menu(username)
            elif choice == '3':
                break
            else:
                print("잘못된 입력입니다. 다시 시도하세요.")
        except ValueError as e:
            print(f"잘못된 입력입니다: {e}")
        except InsufficientFundsError as e:
            print(f"오류: {e}")
        except NegativeAmountError as e:
            print(f"오류: {e}")
        except UserNotFoundError as e:
            print(f"오류: {e}")
        except Exception as e:
            print(f"알 수 없는 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    main()

위 코드로 정상 실행이 안되는 경우 아래 명령어로 실행해보세요

#import 경로 문제로 인해 실행에 오류가 발생할 수 있습니다.

```
python -m banking_system.main
```